In [1]:
#imports here
import time
import csv
import json, requests
import re
import itertools

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

C:\Users\moham\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Helper functions 

In [2]:
def get_url_from_pagename(pagename):
    return "https://www.facebook.com/"+pagename

In [3]:
def scroll_end_of_pubs(driver):
    SCROLL_PAUSE_TIME = 2

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
def login_(driver):
    with open('facebook_credentials.txt') as file:
        EMAIL = file.readline().split('"')[1]
        PASSWORD = file.readline().split('"')[1]
    
    #open the webpage
    driver.get("http://www.facebook.com")

    #target username
    username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
    password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))
    #enter username and password
    username.clear()
    username.send_keys(EMAIL)
    password.clear()
    password.send_keys(PASSWORD)
    #target the validation & login button and click it
    validation = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(( \
                        By.CSS_SELECTOR, "button[class='_42ft _4jy0 _9o-t _4jy3 _4jy1 selected _51sy']"))).click()

    button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable(( \
                                                    By.CSS_SELECTOR, "button[type='submit']"))).click()
    time.sleep(3)

In [5]:
def get_urls_to_scrape_from_page(driver, REQUEST_LINK, keyword):
    PAUSE_TIME = 3
    #go to the page to scrape(ie: France 24)
    driver.get(REQUEST_LINK)
    time.sleep(PAUSE_TIME)

    #click page search btn 
    page_search_btn=driver.find_element_by_class_name("h676nmdw").find_element_by_xpath("//div[@aria-label='Rechercher']")
    page_search_btn.click()

    #search for keyword
    page_search_box = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Rechercher dans cette Page']")))
    page_search_box.clear()
    page_search_box.send_keys(keyword)
    time.sleep(PAUSE_TIME)
    page_search_box.send_keys(Keys.ENTER)
    time.sleep(PAUSE_TIME)

    #Scroll to the end 
    scroll_end_of_pubs(driver)
    time.sleep(PAUSE_TIME)
    
    #get all href urls for posts 
    anchors = driver.find_elements_by_tag_name('a')
    urls_to_scrape = [a.get_attribute('href') for a in anchors]
    urls_to_scrape = [url for url in urls_to_scrape if url.startswith(f'{REQUEST_LINK}/posts/')]
    urls_to_scrape = [url.replace('www','mbasic') for url in urls_to_scrape]
    #return urls to scrape
    return urls_to_scrape

In [6]:
def verify_button_exists(driver):
    try:
        #identify btn element
        more_comments_btn = driver.find_element_by_css_selector("div[id^=see_next_]")  #^= starts with
        #url of 'more comments'
        anchors = driver.find_elements_by_tag_name('a')
        url_more_comments = [a.get_attribute('href') for a in anchors]
        url_more_comments = [url for url in url_more_comments if url.startswith('https://mbasic.facebook.com/story.php?story_fbid=')][0]
        
        return True, more_comments_btn, url_more_comments
    #NoSuchElementException thrown if not present
    except NoSuchElementException:
        return False, False, False

In [7]:
def scraping_one_post(driver, url, author):
    PAUSE_TIME = 3
    LIMIT_COMMENTS = 30
    dict_=dict()
    comments_dict=dict()
    
    #go to post
    driver.get(url)
    time.sleep(PAUSE_TIME)
    
    #get author
    publication_author = author
    #get title
    text = driver.find_elements_by_tag_name('p')
    title = [p.text for p in text][0]
    #get photo
    photos = driver.find_elements_by_tag_name('img')
    photos = [p.get_attribute('src') for p in photos if (p.get_attribute('role')=='presentation' and p.get_attribute('class')=='r')]
    photos = [p for p in photos if p.startswith('https://external')]
    photo = ''
    if len(photos) > 0:
        photo = photos[0]
    #get publication date 
    dates = driver.find_elements_by_tag_name('abbr')
    dates = [p.text for p in dates]
    publication_date = dates[0]
    
    
    #scrap comments 
    comments = []
    comments_dates = []
    comments_names = []
    
    while (verify_button_exists(driver)[0] and len(comments)<LIMIT_COMMENTS):
        more_comments_btn = verify_button_exists(driver)[1]
        url_more_comments = verify_button_exists(driver)[2]
        #dates
        page_dates = driver.find_elements_by_tag_name('abbr')
        page_dates = [p.text for p in page_dates]
        page_comments_dates = page_dates[1:]
        #author 
        page_comments_names = driver.find_elements_by_tag_name('h3')
        page_comments_names = [name.text for name in page_comments_names][2:]
        #comment
        #page_comments = driver.find_elements_by_class_name("br")
        page_comments = driver.find_elements_by_class_name("eg")
        page_comments = [comment.text for comment in page_comments]
        #append to lists
        for i in page_comments:
            comments.append(i)
        for i in page_comments_dates:
            comments_dates.append(i)
        for i in page_comments_names:
            comments_names.append(i)
        #go to more comments
        #more_comments_btn.click()
        driver.get(url_more_comments)
        time.sleep(2)

    #create comments dict
    list_comment_id = [f'comment {i+1}' for i in range(len(comments))]
    list_dict_comments_data = []
    for i in range(min(len(comments_names),len(comments),len(comments_dates))):
        d_temp={}
        d_temp['author'] = comments_names[i]
        d_temp['comment'] = comments[i]
        d_temp['date'] = comments_dates[i]
        list_dict_comments_data.append(d_temp)
    
    comments_dict = dict(zip(list_comment_id, list_dict_comments_data))
    #create output dict 
    dict_['publication author'] = publication_author
    dict_['title'] = title
    dict_['photo'] = photo
    dict_['publication date'] = publication_date
    dict_['comments'] = comments_dict
    
    return dict_

## Main function

In [8]:
chrome_options = Options()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")

In [9]:
pages_to_scrape = ["FRANCE24", "lemonde.fr", "franceinfo"]  
keyword = "Jacques chirac décès"
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
#login 
login_(driver)

In [10]:
big_dict = []
n_posts_limit = 25
#go to page + search for keyword + return urls to scrape
for page_to_scrape in pages_to_scrape:
    REQUEST_LINK = get_url_from_pagename(page_to_scrape)
    urls_to_scrape = get_urls_to_scrape_from_page(driver, REQUEST_LINK, keyword)
    print('Page = '+str(page_to_scrape),'Nb of urls = '+str(len(urls_to_scrape)), urls_to_scrape, sep='\n')
    #scrapping + making dict from results 
    n_start = len(big_dict)
    n_end = len(big_dict)+min(n_posts_limit,len(urls_to_scrape)) 
    n_scrape = 0 #counter for urls_to_scrape
    for i in range(n_start,n_end):
        #append two dicts to create id for mongo
        d_1 = {"_id":i+1}
        d_2 = scraping_one_post(driver, urls_to_scrape[n_scrape], page_to_scrape.split('.')[0])
        n_scrape+=1
        d_tmp= dict(d_1)
        d_tmp.update(d_2)
        big_dict.append(d_tmp)
driver.close()

Page = FRANCE24
Nb of urls = 34
['https://mbasic.facebook.com/FRANCE24/posts/10157122140041936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122326086936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122078016936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122516426936', 'https://mbasic.facebook.com/FRANCE24/posts/10157123831226936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122345146936', 'https://mbasic.facebook.com/FRANCE24/posts/10157427165806936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122109541936', 'https://mbasic.facebook.com/FRANCE24/posts/10157126083221936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122334311936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122901531936', 'https://mbasic.facebook.com/FRANCE24/posts/10157123020366936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122849001936', 'https://mbasic.facebook.com/FRANCE24/posts/10157122407256936', 'https://mbasic.facebook.com/FRANCE24/posts/10158871707901936', 'https:

## Mongo DB

#### Scraping DB

In [13]:
import pymongo
from pymongo import MongoClient
client = #ENTER YOUR MONGO DB CLIENT HERE
db = client["test"]
collection = db["test"]

In [ ]:
collection.insert_many([post for post in big_dict])